In [1]:
import re
import pandas as pd
from typing import  Literal,List,Any
from langchain_core.tools import tool
from langgraph.types import Command
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict, Annotated
from langchain_core.prompts.chat import ChatPromptTemplate
from langgraph.graph import START, StateGraph,END
from langgraph.prebuilt import create_react_agent
# from langchain.agents import create_react_agent
from pydantic import BaseModel, Field, field_validator
from langchain_core.messages import HumanMessage,AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


In [1]:
from langchain_ollama import ChatOllama

In [2]:
llm = ChatOllama(
    model = "gemma3")

In [3]:
messages = [
    ("system", "You are a helpful translator."),
    ("human", "I love programming."),
]

In [5]:
llm.invoke(messages)

AIMessage(content="That’s fantastic! It’s a really rewarding and fascinating field. What kind of programming do you enjoy? Do you have a favorite language or project you're working on? 😊 \n\nI’m here to help if you want to talk about it, need a little explanation, or just want to share your enthusiasm!", additional_kwargs={}, response_metadata={'model': 'gemma3', 'created_at': '2025-04-19T06:06:09.580117Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10985439958, 'load_duration': 8227776708, 'prompt_eval_count': 24, 'prompt_eval_duration': 624571333, 'eval_count': 68, 'eval_duration': 2099963125, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-4e6d252e-2a23-4e2a-b722-75e909f00771-0', usage_metadata={'input_tokens': 24, 'output_tokens': 68, 'total_tokens': 92})

In [ ]:
load_dotenv()

True

In [ ]:
import os
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
llm_model=ChatOpenAI(model="gpt-4o")

In [ ]:
# llm_model.invoke("hey what is the capital of india")

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_ollama.llms import OllamaLLM

# template = """Question: {question}

# Answer: Let's think step by step."""

# prompt = ChatPromptTemplate.from_template(template)

# model = OllamaLLM(model="gemma3")

# chain = prompt | model

# chain.invoke({"question": "What is LangChain?"})

In [ ]:
class DateTimeModel(BaseModel):
    date: str = Field(description=" date format for reservation of appointment", pattern=r'^\d{2}-\d{2}-\d{4} \d{2}:\d{2}$')

    @field_validator("date")
    def check_format_date(cls, date_time_value):
        if not re.match(r'^\d{2}-\d{2}-\d{4} \d{2}:\d{2}$', date_time_value):  # Ensures 'DD-MM-YYYY HH:MM' format
            raise ValueError("The date should be in format 'DD-MM-YYYY HH:MM'")
        return date_time_value

In [ ]:
class DateModel(BaseModel):
    date: str = Field(description="Properly formatted date", pattern=r'^\d{2}-\d{2}-\d{4}$')

    @field_validator("date")
    def check_format_date(cls, date_value):
        if not re.match(r'^\d{2}-\d{2}-\d{4}$', date_value):  # Ensures DD-MM-YYYY format
            raise ValueError("The date must be in the format 'DD-MM-YYYY'")
        return date_value

In [ ]:
class IdentificationNumberModel(BaseModel):
    id: int = Field(description="Identification number (7 or 8 digits long)")
    @field_validator("id")
    def check_format_id(cls, id_number):
        if not re.match(r'^\d{7,8}$', str(id_number)):  # Convert to string before matching
            raise ValueError("The ID number should be a 7 or 8-digit number")
        return id_number

In [ ]:
@tool
def check_availability_by_doctor(desired_date:DateModel, 
                                 doctor_name:Literal['kevin anderson','robert martinez','susan davis','daniel miller',\
                                                     'sarah wilson','michael green','lisa brown','jane smith',\
                                                        'emily johnson','john doe']):
    """
    Checking the database if we have availability for the specific doctor.
    The parameters should be mentioned by the user in the query
    """
    df = pd.read_csv(r"../data/doctor_availability.csv")
    
    #print(df)
    
    df['date_slot_time'] = df['date_slot'].apply(lambda input: input.split(' ')[-1])
    
    rows = list(df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date)&(df['doctor_name'] == doctor_name)&(df['is_available'] == True)]['date_slot_time'])

    if len(rows) == 0:
        output = "No availability in the entire day"
    else:
        output = f'This availability for {desired_date.date}\n'
        output += "Available slots: " + ', '.join(rows)

    return output

In [ ]:
# Example usage:
date_instance = DateModel(date="03-09-2024")
print(date_instance)

date='03-09-2024'


In [ ]:
print(check_availability_by_doctor.invoke({"desired_date": date_instance, "doctor_name": "kevin anderson"}))

This availability for 03-09-2024
Available slots: 08:00, 08:30, 11:30, 12:00, 12:30, 13:00, 14:00, 14:30, 15:00, 15:30, 16:00, 16:30


In [ ]:
@tool
def check_availability_by_specialization(desired_date:DateModel,
                                          specialization:Literal["general_dentist", "cosmetic_dentist", "prosthodontist",\
                                                                                         "pediatric_dentist","emergency_dentist","oral_surgeon","orthodontist"]):
    """
    Checking the database if we have availability for the specific specialization.
    The parameters should be mentioned by the user in the query
    """
    #Dummy data
    df = pd.read_csv(r"../data/doctor_availability.csv")
    df['date_slot_time'] = df['date_slot'].apply(lambda input: input.split(' ')[-1])
    rows = df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date) & (df['specialization'] == specialization) & (df['is_available'] == True)].groupby(['specialization', 'doctor_name'])['date_slot_time'].apply(list).reset_index(name='available_slots')

    if len(rows) == 0:
        output = "No availability in the entire day"
    else:
        def convert_to_am_pm(time_str):
            # Split the time string into hours and minutes
            time_str = str(time_str)
            hours, minutes = map(int, time_str.split(":"))
            
            # Determine AM or PM
            period = "AM" if hours < 12 else "PM"
            
            # Convert hours to 12-hour format
            hours = hours % 12 or 12
            
            # Format the output
            return f"{hours}:{minutes:02d} {period}"
        output = f'This availability for {desired_date.date}\n'
        for row in rows.values:
            output += row[1] + ". Available slots: \n" + ', \n'.join([convert_to_am_pm(value)for value in row[2]])+'\n'

    return output

In [ ]:
# Example usage:
date_instance = DateModel(date="03-09-2024")
print(date_instance)

date='03-09-2024'


In [ ]:
print(check_availability_by_specialization.invoke({"desired_date": date_instance, "specialization": "orthodontist"}))

This availability for 03-09-2024
kevin anderson. Available slots: 
8:00 AM, 
8:30 AM, 
11:30 AM, 
12:00 PM, 
12:30 PM, 
1:00 PM, 
2:00 PM, 
2:30 PM, 
3:00 PM, 
3:30 PM, 
4:00 PM, 
4:30 PM



In [ ]:
@tool
def set_appointment(desired_date:DateTimeModel,
                    id_number:IdentificationNumberModel, 
                    doctor_name:Literal['kevin anderson','robert martinez','susan davis',
                                        'daniel miller','sarah wilson','michael green','lisa brown',
                                        'jane smith','emily johnson','john doe']):
    """
    Set appointment or slot with the doctor.
    The parameters MUST be mentioned by the user in the query.
    """
    df = pd.read_csv(r"../data/doctor_availability.csv")
   
    from datetime import datetime
    def convert_datetime_format(dt_str):
        # Parse the input datetime string
        #dt = datetime.strptime(dt_str, "%Y-%m-%d %H:%M")
        dt = datetime.strptime(dt_str, "%d-%m-%Y %H:%M")
        
        # Format the output as 'DD-MM-YYYY H.M' (removing leading zero from hour only)
        return dt.strftime("%d-%m-%Y %#H.%M")
    
    case = df[(df['date_slot'] == convert_datetime_format(desired_date.date))&(df['doctor_name'] == doctor_name)&(df['is_available'] == True)]
    if len(case) == 0:
        return "No available appointments for that particular case"
    else:
        df.loc[(df['date_slot'] == convert_datetime_format(desired_date.date))&(df['doctor_name'] == doctor_name) & (df['is_available'] == True), ['is_available','patient_to_attend']] = [False, id_number.id]
        df.to_csv(f"../data/doctor_availability.csv", index = False)

        return "Succesfully done"

In [ ]:
Date = DateTimeModel(date="07-08-2024 08:30")
Date

DateTimeModel(date='07-08-2024 08:30')

In [ ]:
IDNumber = IdentificationNumberModel(id=1000097)
IDNumber

IdentificationNumberModel(id=1000097)

In [ ]:
print(set_appointment.invoke({"desired_date":Date,"id_number":IDNumber,"doctor_name":"john doe"}))

No available appointments for that particular case


In [ ]:
@tool
def cancel_appointment(date:DateTimeModel, id_number:IdentificationNumberModel, doctor_name:Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Canceling an appointment.
    The parameters MUST be mentioned by the user in the query.
    """
    df = pd.read_csv(r"../data/doctor_availability.csv")
    case_to_remove = df[(df['date_slot'] == date.date)&(df['patient_to_attend'] == id_number.id)&(df['doctor_name'] == doctor_name)]
    if len(case_to_remove) == 0:
        return "You don´t have any appointment with that specifications"
    else:
        df.loc[(df['date_slot'] == date.date) & (df['patient_to_attend'] == id_number.id) & (df['doctor_name'] == doctor_name), ['is_available', 'patient_to_attend']] = [True, None]
        df.to_csv(f"../data/doctor_availability.csv", index = False)

        return "Successfully cancelled"

In [ ]:
Date = DateTimeModel(date="07-08-2024 08:30")
Date

DateTimeModel(date='07-08-2024 08:30')

In [ ]:
IDNumber = IdentificationNumberModel(id=1000097)
IDNumber

IdentificationNumberModel(id=1000097)

In [ ]:
print(cancel_appointment.invoke({"date": Date,"id_number":IDNumber,"doctor_name":"john doe"}))

You don´t have any appointment with that specifications


In [ ]:
## Tools For Booking agent
@tool
def reschedule_appointment(old_date:DateTimeModel, new_date:DateTimeModel, id_number:IdentificationNumberModel, 
                           doctor_name:Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Rescheduling an appointment.
    The parameters MUST be mentioned by the user in the query.
    """
    #Dummy data
    df = pd.read_csv(r"../data/doctor_availability.csv")
    available_for_desired_date = df[(df['date_slot'] == new_date.date)&(df['is_available'] == True)&(df['doctor_name'] == doctor_name)]
    if len(available_for_desired_date) == 0:
        return "Not available slots in the desired period"
    else:
        cancel_appointment.invoke({'date':old_date, 'id_number':id_number, 'doctor_name':doctor_name})
        set_appointment.invoke({'desired_date':new_date, 'id_number': id_number, 'doctor_name': doctor_name})
        return "Successfully rescheduled for the desired time"

In [ ]:
Old_Date_Time = DateTimeModel(date="05-08-2024 08:30")
Old_Date_Time

DateTimeModel(date='05-08-2024 08:30')

In [ ]:
New_Date_Time = DateTimeModel(date="28-03-2024 14:30")
New_Date_Time

DateTimeModel(date='28-03-2024 14:30')

In [ ]:
IDNumber = IdentificationNumberModel(id=1000082)
IDNumber

IdentificationNumberModel(id=1000082)

In [ ]:
print(reschedule_appointment.invoke({"old_date": Old_Date_Time,"new_date": New_Date_Time,"id_number":IDNumber, "doctor_name": "kevin anderson"}))

Not available slots in the desired period


In [ ]:
class Router(TypedDict):
    next: Literal["information_node","booking_node","FINISH"]
    reasoning: str

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[Any], add_messages]
    id_number: int
    next:str
    query: str
    current_reasoning: str

In [ ]:
members_dict = {'information_node':'specialized agent to provide information related to availability of doctors or any FAQs related to hospital.',
                'booking_node':'specialized agent to only to book, cancel or reschedule appointment'}

In [ ]:
options = list(members_dict.keys()) + ["FINISH"]

In [ ]:
worker_info = '\n\n'.join([f'WORKER: {member} \nDESCRIPTION: {description}' for member, description in members_dict.items()]) + \
      '\n\nWORKER: FINISH \nDESCRIPTION: If User Query is answered and route to Finished'

In [ ]:
system_prompt = (
    "You are a supervisor tasked with managing a conversation between following workers. "
    "### SPECIALIZED ASSISTANT:\n"
    f"{worker_info}\n\n"
    "Your primary role is to help the user make an appointment with the doctor and provide updates on FAQs and doctor's availability. "
    "If a customer requests to know the availability of a doctor or to book, reschedule, or cancel an appointment, "
    "delegate the task to the appropriate specialized workers. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
    "UTILIZE last conversation to assess if the conversation should end you answered the query, then route to FINISH "
     )

In [ ]:
state  = AgentState

In [ ]:
messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": f"user's identification number is {state['id_number']}"},]

In [7]:
import dask.dataframe as dd

In [8]:
df = dd.read_csv(r"../data/doctor_availability.csv")

,date_slot,specialization,doctor_name,is_available,patient_to_attend,date_slot_time
0,05-08-2024 08:00,general_dentist,john doe,True,NaN,08:00
1,05-08-2024 08:30,general_dentist,john doe,False,1000082.0,08:30
2,05-08-2024 09:00,general_dentist,john doe,False,1000048.0,09:00
3,05-08-2024 09:30,general_dentist,john doe,False,1000036.0,09:30
4,05-08-2024 10:00,general_dentist,john doe,False,1000024.0,10:00


In [ ]:
# def supervisor_node(state:AgentState) -> Command[Literal['information_node', 'booking_node', '__end__']]:
    
#     print("Below is the state after kickstarting of agent",state)
    
#     messages = [
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": f"user's identification number is {state['id_number']}"},
#     ] + state["messages"]
    
    
#     print("this is a new message",messages)
    
#     # query = state['messages'][-1].content if state["messages"] else ""
#     query = ''
#     if len(state['messages']) == 1:
#         query = state['messages'][0].content
      
    
#     print("This is a new query",query)
    
#     response = llm_model.with_structured_output(Router).invoke(messages)
    
#     goto = response["next"]
    
    
#     print("the task for next actions",goto)
    
    
#     print(response["this is reasoning action"])
          
#     if goto == "FINISH":
#         goto = END
        

#     print("this is task state",state)
    
#     if query:
#         return Command(goto=goto, update={'next': goto, 
#                                           'query': query, 
#                                           'current_reasoning': response["reasoning"],
#                                           'messages': [HumanMessage(content=f"user's identification number is {state['id_number']}")]
#                         })
#     return Command(goto=goto, update={'next': goto, 
#                                       'current_reasoning': response["reasoning"]}
#                    )






def supervisor_node(state:AgentState) -> Command[Literal['information_node', 'booking_node', '__end__']]:
    print("**************************below is my state right after entering****************************")
    print(state)
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"user's identification number is {state['id_number']}"},
    ] + state["messages"]
    
    print("***********************this is my message*****************************************")
    print(messages)
    
    # query = state['messages'][-1].content if state["messages"] else ""
    query = ''
    if len(state['messages']) == 1:
        query = state['messages'][0].content
      
    print("************below is my query********************")    
    print(query)
    
    response = llm_model.with_structured_output(Router).invoke(messages)
    
    goto = response["next"]
    
    print("********************************this is my goto*************************")
    print(goto)
    
    print("********************************")
    print(response["reasoning"])
          
    if goto == "FINISH":
        goto = END
        
    print("**************************below is my state****************************")
    print(state)
    
    if query:
        return Command(goto=goto, update={'next': goto, 
                                          'query': query, 
                                          'current_reasoning': response["reasoning"],
                                          'messages': [HumanMessage(content=f"user's identification number is {state['id_number']}")]
                        })
    return Command(goto=goto, update={'next': goto, 
                                      'current_reasoning': response["reasoning"]}
                   )




In [ ]:
# def information_node(state:AgentState) -> Command[Literal['supervisor']]:


#     system_prompt_template = "You are specialised agent to provide information related to availablity of doctors or any any FAQs related to hospital based on the query.\
#           You have access to the tool.\n  Make sure to ask user politely if you need any further information to execute the tool.\
#             \n For your information, Always consider current year is 2024"
    

#     system_prompt = ChatPromptTemplate.from_messages(
#         [
#             (
#                 "system", system_prompt_template
#             ),
#             (
#                 "human message","{messages}"
#             )
#         ]
#     )

#     information_agent =  create_react_agent(model=llm_model,tools=[check_availability_by_specialization,
#                                                                check_availability_by_specialization],
#                                                                prompt=system_prompt)
    
#     result = information_agent.invoke(state)

#     return Command(
#         update={
#             "messages":state["messages"] + [
#                 AIMessage(content=result["messages"][-1].content,
#                           name="information_node")
#             ]
#         },goto="supervisor"
#     )



def information_node(state:AgentState) -> Command[Literal['supervisor']]:
    print("*****************called information node************")
    try:
        system_prompt = "You are specialized agent to provide information related to availability of doctors or any FAQs related to hospital based on the query. You have access to the tool.\n Make sure to ask user politely if you need any further information to execute the tool.\n For your information, Always consider current year is 2024."
        
        system_prompt = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        system_prompt
                    ),
                    (
                        "placeholder", 
                        "{messages}"
                    ),
                ]
            )
        
        information_agent = create_react_agent(model = llm_model,
                                            tools=[check_availability_by_doctor,
                                                    check_availability_by_specialization] ,prompt=system_prompt)
        
        result = information_agent.invoke(state)
        
        return Command(
            update={
                "messages": state["messages"] + [
                    AIMessage(content=result["messages"][-1].content, name="information_node")
                ]
            },
            goto="supervisor",
        )
    except Exception as e:
        print(e)



In [ ]:
def booking_node(state:AgentState)->Command[Literal['supervisor']] :
    system_prompt_template = "You are specialized agent to set, cancel or reschedule appointment based on the query. You have access to the tool.\n Make sure to ask user politely if you need any further information to execute the tool.\n For your information, Always consider current year is 2024."
    
    system_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    system_prompt_template
                ),
                (
                        "placeholder", 
                        "{messages}"
                ),
            ]
        )
    booking_agent = create_react_agent(model=llm_model,tools=[set_appointment,cancel_appointment,reschedule_appointment],prompt=system_prompt)

    result = booking_agent.invoke(state)
    
    return Command(
        update={
            "messages": state["messages"] + [
                AIMessage(content=result["messages"][-1].content, name="booking_node")
            ]
        },
        goto="supervisor",
    )

In [ ]:
graph_task = StateGraph(AgentState)
graph_task.add_node("supervisor",supervisor_node)
graph_task.add_node("information_node",information_node)
graph_task.add_node("booking_node",booking_node)
graph_task.add_edge(START,"supervisor")
app =  graph_task.compile()

In [ ]:
import langgraph

graph = app.get_graph()
print(graph)

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'supervisor': Node(id='supervisor', name='supervisor', data=supervisor(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'information_node': Node(id='information_node', name='information_node', data=information_node(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'booking_node': Node(id='booking_node', name='booking_node', data=booking_node(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), '__end__': Node(id='__end__', name='__end__', data=<class 'langchain_core.utils.pydantic.LangGraphOutput'>, metadata=None)}, edges=[Edge(source='__start__', target='supervisor', data=None, conditional=False), Edge(source='supervisor', target='information_node', data=None, conditional=True), Edg

In [ ]:
# from IPython.display import Image, display
# display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
inputs = [
        HumanMessage(content='can you check and make a booking if general dentist available on 8 August 2024 at 8 PM?')
    ]

In [ ]:
state = {'messages': inputs,'id_number':10232303}

In [ ]:
result = app.invoke(state,{"recursion_limit": 20})

**************************below is my state right after entering****************************
{'messages': [HumanMessage(content='can you check and make a booking if general dentist available on 8 August 2024 at 8 PM?', additional_kwargs={}, response_metadata={}, id='baa3ff76-81db-41e1-9c8d-0573c6b83768')], 'id_number': 10232303}
***********************this is my message*****************************************
[{'role': 'system', 'content': "You are a supervisor tasked with managing a conversation between following workers. ### SPECIALIZED ASSISTANT:\nWORKER: information_node \nDESCRIPTION: specialized agent to provide information related to availability of doctors or any FAQs related to hospital.\n\nWORKER: booking_node \nDESCRIPTION: specialized agent to only to book, cancel or reschedule appointment\n\nWORKER: FINISH \nDESCRIPTION: If User Query is answered and route to Finished\n\nYour primary role is to help the user make an appointment with the doctor and provide updates on FAQs 

GraphRecursionError: Recursion limit of 20 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [ ]:
result['messages'][-1]

HumanMessage(content="user's identification number is 10232303", additional_kwargs={}, response_metadata={}, id='285b8f16-1feb-4fbf-89af-02ab87a1ab87')

In [ ]:
#Please book an appointment if general dentist available on 8 August 2024 at 8 PM

#Please book an appointment with Emily Johnson on 08-08-2024 at 4:30 PM
